# Web Scrapping

## Introduction

<b><i>Data source:</b></i>
https://boardgamegeek.com/browse/boardgame  <b>OR</b> <br>
--> https://boardgamegeek.com/browse/boardgame/page/1

(An inner page looks like: https://boardgamegeek.com/boardgame/224517/brass-birmingham)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep #allows to input a delay between 'clicking' from one page to another so our behavior looks like human

## Connection to my 'dummy' account

To avoid the connecting issue (otherwise we get limited to 1000 rows maximum), we use a method proposed here to keep our cookies and headers from our connection. After a few days, the info might naturally change so one should do the method again.

<i><b>Link:</i></b> https://stackoverflow.com/questions/23102833/how-to-scrape-a-website-which-requires-login-using-python-and-beautifulsoup/23103784#23103784
<br>
--> Apply the second solution with cURL and https://curlconverter.com/

<i><b>PS</b></i>: There is maybe an alternative way to get to the same thing, but we won't get there  <br>
https://stackoverflow.com/questions/13925983/login-to-website-using-urllib2-python-2-7

In [3]:
import requests

cookies = {
    'FCCDCF': '%5Bnull%2Cnull%2Cnull%2C%5B%22CP9Dx0AP9Dx0AEsACBFRAwEgAAAAAEPgACiQAAAOhQD2F2K2kKFkPCmQWYAQBCijYEAhQAAAAkCBIAAgAUgQAgFIIAgAIFAAAAAAAAAQEgCQAAQABAAAIACgAAAAAAIAAAAAAAQQAAAAAIAAAAAAAAEAAAAAAAQAAAAIAABEgCAAQQAEAAAAAAAQAAAAAAAAAAABAAA%22%2C%222~~dv.2072.70.89.93.108.122.149.196.2253.2299.259.2357.311.313.323.2373.338.358.2415.415.449.2506.2526.486.494.495.2568.2571.2575.540.574.2624.609.2677.864.981.1029.1048.1051.1095.1097.1126.1205.1211.1276.1301.1365.1415.1423.1449.1570.1577.1598.1651.1716.1735.1753.1765.1870.1878.1889.1896.1958%22%2C%22AA367782-2909-488F-BBAF-1AB8DC80A850%22%5D%5D',
    'bggusername': 'Irontest',
    'bggpassword': '662cslfyvmae8zyutygw8gmiqzwlv3gc',
    'cc_cookie': '%7B%22level%22%3A%5B%22necessary%22%5D%2C%22revision%22%3A1%2C%22rfc_cookie%22%3Atrue%7D',
    'SessionID': '3fdc094672b845353e20748c0e495a9f968d3905u3859721',
    '__eoi': 'ID=473c38d32d91b92c:T=1713107946:RT=1713683802:S=AA-AfjZMK2Nbd7qdOjIfINGC-W4n',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'fr-FR,fr;q=0.9,de;q=0.8',
    'cache-control': 'max-age=0',
    # 'cookie': 'FCCDCF=%5Bnull%2Cnull%2Cnull%2C%5B%22CP9Dx0AP9Dx0AEsACBFRAwEgAAAAAEPgACiQAAAOhQD2F2K2kKFkPCmQWYAQBCijYEAhQAAAAkCBIAAgAUgQAgFIIAgAIFAAAAAAAAAQEgCQAAQABAAAIACgAAAAAAIAAAAAAAQQAAAAAIAAAAAAAAEAAAAAAAQAAAAIAABEgCAAQQAEAAAAAAAQAAAAAAAAAAABAAA%22%2C%222~~dv.2072.70.89.93.108.122.149.196.2253.2299.259.2357.311.313.323.2373.338.358.2415.415.449.2506.2526.486.494.495.2568.2571.2575.540.574.2624.609.2677.864.981.1029.1048.1051.1095.1097.1126.1205.1211.1276.1301.1365.1415.1423.1449.1570.1577.1598.1651.1716.1735.1753.1765.1870.1878.1889.1896.1958%22%2C%22AA367782-2909-488F-BBAF-1AB8DC80A850%22%5D%5D; bggusername=Irontest; bggpassword=662cslfyvmae8zyutygw8gmiqzwlv3gc; cc_cookie=%7B%22level%22%3A%5B%22necessary%22%5D%2C%22revision%22%3A1%2C%22rfc_cookie%22%3Atrue%7D; SessionID=3fdc094672b845353e20748c0e495a9f968d3905u3859721; __eoi=ID=473c38d32d91b92c:T=1713107946:RT=1713683802:S=AA-AfjZMK2Nbd7qdOjIfINGC-W4n',
    'dnt': '1',
    'priority': 'u=0, i',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
}

response = requests.get('https://boardgamegeek.com/browse/boardgame/page/1', cookies=cookies, headers=headers)

## Scraping the data

In [4]:
# Function to scrape data from a single page
def scrape_page(url):
    response = requests.get(url, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr', {'id': lambda x: x and x.startswith('row_')})
    
    data = []
    for row in rows:
        cells = row.find_all('td')
        rank = cells[0].text.strip()
        title_cell = cells[2]
        title_link = title_cell.find('a', class_='primary')
        title = title_link.text.strip() if title_link else 'N/A'
        game_url = title_link['href'] if title_link else 'N/A'
        full_game_url = f'https://boardgamegeek.com{game_url}' if game_url != 'N/A' else 'N/A'
        geek_rating = cells[4].text.strip() if cells[4].text.strip() else 'N/A'
        avg_rating = cells[5].text.strip() if cells[5].text.strip() else 'N/A'
        num_voters = cells[6].text.strip() if cells[6].text.strip() else 'N/A'
        data.append([rank, title, full_game_url, geek_rating, avg_rating, num_voters])
    
    sleep(randint(3,10))
    
    return data


In [25]:
# Scrape all pages and concatenate data into a single list
all_data = []
base_url = 'https://boardgamegeek.com/browse/boardgame/page/'
num_pages = 1357

for page_num in range(1000, num_pages + 1):
    page_url = f'{base_url}{page_num}'
    page_data = scrape_page(page_url)
    all_data.extend(page_data)

# Create DataFrame
df = pd.DataFrame(all_data, columns=['Board Game Rank', 'Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'])

In [9]:
# Print the DataFrame
display(df)

,Board Game Rank,Title,URL,Geek Rating,Avg Rating,Num Voters
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.417,8.60,44659
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.382,8.53,53075
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.358,8.59,61674
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.333,8.54,41856
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.243,8.61,23345
...,...,...,...,...,...,...
2195,2196,Subbuteo,https://boardgamegeek.com/boardgame/3720/subbuteo,6.152,6.99,1840
2196,2197,Agents of SMERSH,https://boardgamegeek.com/boardgame/111105/age...,6.152,6.96,1679
2197,2198,The Princes of Machu Picchu,https://boardgamegeek.com/boardgame/36811/prin...,6.152,6.95,1708
2198,2199,Take it Easy!,https://boardgamegeek.com/boardgame/128/take-i...,6.151,6.61,3203


# In case of data retrieving (VS Code  Stop/Crash)

## Saving the succeeded web scraped things

--> Only run this to save the rows for which the scraping succeeded before stoping, so we can save them as a dataframe we'll concatenate in the end 

In [16]:
'''
all_data
df_save = pd.DataFrame(all_data, columns=['Board Game Rank', 'Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'])
display(df_save)
'''

,Board Game Rank,Title,URL,Geek Rating,Avg Rating,Num Voters
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.417,8.60,44659
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.382,8.53,53075
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.358,8.59,61674
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.333,8.54,41856
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.243,8.61,23345
...,...,...,...,...,...,...
2195,2196,Subbuteo,https://boardgamegeek.com/boardgame/3720/subbuteo,6.152,6.99,1840
2196,2197,Agents of SMERSH,https://boardgamegeek.com/boardgame/111105/age...,6.152,6.96,1679
2197,2198,The Princes of Machu Picchu,https://boardgamegeek.com/boardgame/36811/prin...,6.152,6.95,1708
2198,2199,Take it Easy!,https://boardgamegeek.com/boardgame/128/take-i...,6.151,6.61,3203


In [ ]:
'''
# Save DataFrame to a CSV file
df_save.to_csv('web_scraping_pagexxx_to_yyy.csv', index=False)
'''

## Continue the scrapping starting from the last page successfully runned +1

Then run the following code:

In [5]:
base_url = 'https://boardgamegeek.com/browse/boardgame/page/'
num_pages = 1357

In [5]:
# [RE-PRECIZING THINGS]
base_url = 'https://boardgamegeek.com/browse/boardgame/page/'
num_pages = 1357

# Load the previously saved data from the CSV file
previous_data = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page261_to_yyy.csv')

# Get the last page number from the previous data
last_page_scraped = previous_data['Board Game Rank'].max() // 100

# Define the page number to start scraping from
start_page = last_page_scraped + 1
start_page = int(start_page)

# [RUNNING THE SCRAPING]
all_data = []

# Scrape pages starting from page x, x being the last successful page
for page_num in range(start_page, num_pages + 1):
    page_url = f'{base_url}{page_num}'
    page_data = scrape_page(page_url)
    all_data.extend(page_data)

def scrape_page(url):
    response = requests.get(url, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr', {'id': lambda x: x and x.startswith('row_')})   
    data = []

    for row in rows:
        cells = row.find_all('td')
        rank = cells[0].text.strip()
        title_cell = cells[2]
        title_link = title_cell.find('a', class_='primary')
        title = title_link.text.strip() if title_link else 'N/A'
        game_url = title_link['href'] if title_link else 'N/A'
        full_game_url = f'https://boardgamegeek.com{game_url}' if game_url != 'N/A' else 'N/A'
        geek_rating = cells[4].text.strip() if cells[4].text.strip() else 'N/A'
        avg_rating = cells[5].text.strip() if cells[5].text.strip() else 'N/A'
        num_voters = cells[6].text.strip() if cells[6].text.strip() else 'N/A'
        data.append([rank, title, full_game_url, geek_rating, avg_rating, num_voters])
    
    sleep(randint(3,10))
    
    return data

# Concatenate the new data with the previous data
all_data_df = pd.DataFrame(all_data, columns=['Board Game Rank', 'Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'])
all_data_concatenated = pd.concat([previous_data, all_data_df])

# saving the final table to csv
all_data_concatenated.to_csv('web_scraping_pagevcf_to_xxx.csv', index=False)


In [ ]:
df_save = pd.DataFrame(all_data, columns=['Board Game Rank', 'Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'])
display(df_save)

,Board Game Rank,Title,URL,Geek Rating,Avg Rating,Num Voters
0,26103,Pop-Up Pirate!,https://boardgamegeek.com/boardgame/9004/pop-p...,5.284,4.59,595
1,26104,Hengist,https://boardgamegeek.com/boardgame/182875/hen...,5.282,4.78,755
2,26105,Mad Gab,https://boardgamegeek.com/boardgame/764/mad-gab,5.279,5.05,1603
3,26106,Thermopyles,https://boardgamegeek.com/boardgame/141019/the...,5.276,3.92,340
4,26107,Buckaroo!,https://boardgamegeek.com/boardgame/8392/buckaroo,5.272,4.56,602
...,...,...,...,...,...,...
9395,N/A,Nick Game,https://boardgamegeek.com/boardgame/17063/nick...,N/A,N/A,N/A
9396,N/A,StarMarines,https://boardgamegeek.com/boardgame/17064/star...,N/A,2.00,2
9397,N/A,Hack & Sack New Jersey,https://boardgamegeek.com/boardgame/17065/hack...,N/A,N/A,N/A
9398,N/A,Fairy Meat: Clockwork Stomp,https://boardgamegeek.com/boardgameexpansion/1...,N/A,6.47,16


In [ ]:
#saving the final table to csv
df_save.to_csv('web_scraping_page261_to_zzz.csv', index=False)

In [ ]:
# Locating where the file has been saved
import os
print(os.getcwd())

d:\Docs Persos\IRONHACK_BC\FINAL PROJECT\board_games_project


# Final step: merging the dataframes together

## Obtaining the final df

In [19]:
df_1 = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page1_to_561.csv')
df_2 = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page261_to_796.csv')
df_3 = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page261_to_xxx.csv')
df_4 = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page261_to_yyy.csv')
df_5 = pd.read_csv('D:/Docs Persos/IRONHACK_BC/FINAL PROJECT/board_games_project/Web scraping files/web_scraping_page261_to_zzz.csv')

In [20]:
# List of DataFrames
dfs = [df_1, df_2, df_3, df_4]

# Concatenate all DataFrames
concatenated_df = pd.concat(dfs, ignore_index=True)

In [23]:
# reinitialize the rows Index
concatenated_df.reset_index(drop=True, inplace=True)

In [24]:
display (concatenated_df)

,Board Game Rank,Title,URL,Geek Rating,Avg Rating,Num Voters
0,1.0,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.417,8.60,44659.0
1,2.0,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.382,8.53,53075.0
2,3.0,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.358,8.59,61674.0
3,4.0,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.333,8.54,41856.0
4,5.0,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.243,8.61,23345.0
...,...,...,...,...,...,...
278795,NaN,Nick Game,https://boardgamegeek.com/boardgame/17063/nick...,NaN,NaN,NaN
278796,NaN,StarMarines,https://boardgamegeek.com/boardgame/17064/star...,NaN,2.00,2.0
278797,NaN,Hack & Sack New Jersey,https://boardgamegeek.com/boardgame/17065/hack...,NaN,NaN,NaN
278798,NaN,Fairy Meat: Clockwork Stomp,https://boardgamegeek.com/boardgameexpansion/1...,NaN,6.47,16.0


In [27]:
#drop duplicates
concatenated_df.drop_duplicates(subset=['Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'], inplace=True)

# reinitialize the rows Index
concatenated_df.reset_index(drop=True, inplace=True)

#checking now the shape of the df
concatenated_df.shape

(56277, 6)

In [29]:
#Normalizing column names

# Loop through each column
for column in concatenated_df.columns:
    # Normalize column name by converting to lowercase and replacing spaces with underscores
    normalized_column_name = column.lower().replace(' ', '_')
    # Update the column name in the DataFrame
    concatenated_df.rename(columns={column: normalized_column_name}, inplace=True)


In [30]:
concatenated_df.head(0)

,board_game_rank,title,url,geek_rating,avg_rating,num_voters


# Sending to MySQL